In [82]:
# KNN: Multiple Participants

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json

# 1 -- Load Data

In [84]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [85]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [86]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [87]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [88]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [89]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [90]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [91]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [92]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [93]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [94]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [95]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features
# features_to_keep = basic_cols + key_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [96]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [97]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [98]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for training
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i], split_size)
            train_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i])
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)

    y_train = train_data['engagement']
    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])
    y_test = test_data['engagement']
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    # Sort Training Data Together 
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [99]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn import neighbors

# Hyperparameter Combinations to Try:  

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = neighbors.KNeighborsClassifier(n_neighbors=250, weights='distance', algorithm='auto')
    clf.fit(X_train, y_train)
    
    scores = clf.predict_proba(X_test.values)[:,1]
    pred = [round(value) for value in scores]   
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [100]:
all_results = []
for n in range(1, len(part)):
    comb = combinations(part, n)
    print()
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(train_part, test_part, 0)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        '''
        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1

        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_acc = None, 0
        for h in hp_combos:
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_acc = acc
        '''
        best = {}
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        results['hyperparameters'] = json.dumps(best)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
Best HP {}
{'precision_1': 0.7478424641205359, 'accuracy': 0.736816436558396, 'auc': 0.7583493576649587, 'recall_0': 0.490375581199258, 'recall_1': 0.880360350246969, 'f1_1': 0.8087086545154554, 'f1_0': 0.5783491973291661, 'precision_0': 0.704788615352654}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
Best HP {}
{'precision_1': 0.7573691967575534, 'accuracy': 0.7327524152301572, 'auc': 0.739915539187068, 'recall_0': 0.4427991199864613, 'recall_1': 0.879345749080227, 'f1_1': 0.813812307814129, 'f1_0': 0.5266883617942094, 'precision_0': 0.6497930463576159}

Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
Best HP {}
{'precision_1': 0.8316943596699446, 'accuracy': 0.7089271009929765, 'auc': 0.7475876897874192, 'recall_0': 0.6752554835493519, 'recall_1': 0.7241209633201647, 'f1_1': 0.7741886996648079, 'f1_0': 0.5906093307172443, 'precision_0': 0.5248208058891902}

Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
Best HP {}
{'precision_1': 0.8056696671298255, 'accurac

{'precision_1': 0.8143335919829622, 'accuracy': 0.7504089219330855, 'auc': 0.7893773941518822, 'recall_0': 0.6547341274714479, 'recall_1': 0.8010165968365325, 'f1_1': 0.8076202015571148, 'f1_0': 0.6447672748711283, 'precision_0': 0.6350993180261473}


Train: [5, 7, 9] 	Test: [11, 12, 17, 18]
Best HP {}
{'precision_1': 0.78431189926235, 'accuracy': 0.7479833537610808, 'auc': 0.750580034238643, 'recall_0': 0.4726510907683199, 'recall_1': 0.8733846377232467, 'f1_1': 0.8264552030070761, 'f1_0': 0.5399724238403705, 'precision_0': 0.6296565950734134}

Train: [5, 7, 11] 	Test: [9, 12, 17, 18]
Best HP {}
{'precision_1': 0.7009884213499011, 'accuracy': 0.7087018954623779, 'auc': 0.7443727481015013, 'recall_0': 0.43284648073420706, 'recall_1': 0.8942286908278695, 'f1_1': 0.785904255319149, 'f1_0': 0.5444214876033058, 'precision_0': 0.7334947049924357}

Train: [5, 7, 12] 	Test: [9, 11, 17, 18]
Best HP {}
{'precision_1': 0.7424435636739617, 'accuracy': 0.7266107678729038, 'auc': 0.7251015263708627

{'precision_1': 0.8159033948507632, 'accuracy': 0.7702016089632935, 'auc': 0.809388048083145, 'recall_0': 0.6744560838033844, 'recall_1': 0.8249462475683423, 'f1_1': 0.8203999032697373, 'f1_0': 0.6810568903555365, 'precision_0': 0.6877881762154759}

Train: [9, 12, 17] 	Test: [5, 7, 11, 18]
Best HP {}
{'precision_1': 0.8487148625562976, 'accuracy': 0.74551937818852, 'auc': 0.7744461800611777, 'recall_0': 0.608332914509725, 'recall_1': 0.7951656026627381, 'f1_1': 0.8210680514212203, 'f1_0': 0.5595543536046969, 'precision_0': 0.5180176324574167}

Train: [9, 12, 18] 	Test: [5, 7, 11, 17]
Best HP {}
{'precision_1': 0.8315272389947714, 'accuracy': 0.7793634646127017, 'auc': 0.8133851305554565, 'recall_0': 0.6396879790783659, 'recall_1': 0.8457925887802368, 'f1_1': 0.8385992515733968, 'f1_0': 0.6514372302323446, 'precision_0': 0.6636261577322481}

Train: [9, 17, 18] 	Test: [5, 7, 11, 12]
Best HP {}
{'precision_1': 0.848299486175679, 'accuracy': 0.7713376773779458, 'auc': 0.7969392908212732, '

{'precision_1': 0.8291917591125199, 'accuracy': 0.7595810200274226, 'auc': 0.7804382267883074, 'recall_0': 0.5861300975347515, 'recall_1': 0.8313524850641922, 'f1_1': 0.8302707163032785, 'f1_0': 0.5879815100154083, 'precision_0': 0.5898446556920937}

Train: [7, 9, 11, 17] 	Test: [5, 12, 18]
Best HP {}
{'precision_1': 0.8427430771844385, 'accuracy': 0.7639182225225987, 'auc': 0.7611461774639325, 'recall_0': 0.5356187290969899, 'recall_1': 0.8410760709845145, 'f1_1': 0.841908748903912, 'f1_0': 0.5340558566069195, 'precision_0': 0.5325020781379883}

Train: [7, 9, 11, 18] 	Test: [5, 12, 17]
Best HP {}
{'precision_1': 0.7957722812205942, 'accuracy': 0.771390503875969, 'auc': 0.7928499011922481, 'recall_0': 0.5732743487114669, 'recall_1': 0.8757350493731277, 'f1_1': 0.8338410071309095, 'f1_0': 0.6337279254803028, 'precision_0': 0.7084345713293995}

Train: [7, 9, 12, 17] 	Test: [5, 11, 18]
Best HP {}
{'precision_1': 0.8453801568360041, 'accuracy': 0.733562244301578, 'auc': 0.7093282400772166,

{'precision_1': 0.8213401100138321, 'accuracy': 0.7763197551758514, 'auc': 0.7873513158177728, 'recall_0': 0.5887753590996594, 'recall_1': 0.8618152361022041, 'f1_1': 0.841091016898903, 'f1_0': 0.6224170319348779, 'precision_0': 0.6601361447783497}

Train: [5, 11, 12, 17, 18] 	Test: [7, 9]
Best HP {}
{'precision_1': 0.7647007119627318, 'accuracy': 0.7416152275447696, 'auc': 0.8163010877620973, 'recall_0': 0.7261661211129297, 'recall_1': 0.7547169811320755, 'f1_1': 0.7596760461917963, 'f1_0': 0.720619210760056, 'precision_0': 0.7151563995365939}

Train: [7, 9, 11, 12, 17] 	Test: [5, 18]
Best HP {}
{'precision_1': 0.857212594581401, 'accuracy': 0.7545440226807372, 'auc': 0.7228599192529697, 'recall_0': 0.45079017368173996, 'recall_1': 0.8311114617022956, 'f1_1': 0.843960269144505, 'f1_0': 0.4251143573852737, 'precision_0': 0.4022057797012425}

Train: [7, 9, 11, 12, 18] 	Test: [5, 17]
Best HP {}
{'precision_1': 0.8250155780887102, 'accuracy': 0.7846321610780076, 'auc': 0.8165445797632778,

In [101]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'hyperparameters'])
results = results.append(all_results, ignore_index=True, sort=True)

In [102]:
results.head()

,accuracy,auc,f1_0,f1_1,hyperparameters,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.736816,0.758349,0.578349,0.808709,{},0.704789,0.747842,0.490376,0.880360,"7, 9, 11, 12, 17, 18",5
1,0.732752,0.739916,0.526688,0.813812,{},0.649793,0.757369,0.442799,0.879346,"5, 9, 11, 12, 17, 18",7
2,0.708927,0.747588,0.590609,0.774189,{},0.524821,0.831694,0.675255,0.724121,"5, 7, 11, 12, 17, 18",9
3,0.765538,0.808508,0.677351,0.815866,{},0.692712,0.805670,0.662655,0.826324,"5, 7, 9, 12, 17, 18",11
4,0.745847,0.797904,0.641092,0.803268,{},0.625034,0.814741,0.657996,0.792113,"5, 7, 9, 11, 17, 18",12


In [103]:
results.to_csv('results.csv', index=False)